# Evaluation for Redbox RAG chat  <a class="anchor" id="title"></a>

-----------

**Evaluate Redbox RAG chat on one stable, numbered version of these data**

-----------------------

## Table of Contents <a class="anchor" id="toc"></a>
* [Overview](#overview)
* [Metrics](#metrics)
    - [Contextual Precisio]()
    - [Contextual Recall]()
    - [Contextual Relevancy]()
    - [Fathfulness]()
    - [Answer Relevancy]()
    - [Hallucination]()
* [Set version of the evaluation dataset](#setversion)
* [Imports](#imports)
* [Run Redbox Locally](#run-redbox)
* [Get files that correspond to the version of evaluation dataset](#files)
* [Load Evaluation Dataset into test cases](#load-test-cases)

* [Generate `actual_output` using RAG and evaluation dataset](#evaluate)
* [Evaluate RAG pipeline](#five-section)
    - [Retrieval Evaluation Metrics](#six-section)
    - [Generation Evaluation Metrics](#seven-section)
* [#TODO](#todo)

------------

## Overview <a class="anchor" id="overview"></a>

Redbox RAG chat is made up of many components that work together to give the final RAG pipeline.Each component can be optimised, to hopefully improve the over all performance of the RAG pipeline for Redbox tasks. In order to track if changes made are improving or degrading Redbox performance, we need to establish an evaluation framework. The overall RAG pipeline can be broken down into two main parts:

1. Retrieval - searching and returning the most relevant documents to answer a user question
2. Generation - the ouput of the LLM after considering the retrieved documents, any prompts provided and the user question

This notebook tests both the retrieval and generation sides of the RAG pipeline using specific metrics for each, using the `DeepEval` framework.


For consistency across the team, it is important to evaluate Redbox RAG chat on one stable, numbered version of these data.


[Back to top](#title)


-------

## Metrics <a class="anchor" id="metrics"></a>

Retrieval metrics
- Contextual Precision
- Contextual Recall
- Contextual Relevancy

Generation metrics
- Faithfulness
- Answer Relevancy
- Hallucination


### Contextual Precision

The contextual precision metric measures your RAG pipeline's retriever by evaluating whether nodes in your `retrieval_context` that are relevant to the given `input` are ranked higher than irrelevant ones.

### Contextual Recall

The contextual recall metric measures the quality of your RAG pipeline's retriever by evaluating the extent of which the `retrieval_context` aligns with the `expected_output`.

### Contextual Relevancy

The contextual relevancy metric measures the quality of your RAG pipeline's retriever by evaluating the overall relevance of the information presented in your `retrieval_context` for a given `input`.

### Faithfulness

The faithfulness metric measures the quality of your RAG pipeline's generator by evaluating whether the `actual_output` factually aligns with the contents of your `retrieval_context`. `deepeval`'s faithfulness metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score.

##### Required Arguments
To use the `FaithfulnessMetric`, you need to provide the following arguments when creating an LLMTestCase:

- `input`
- `actual_output`
- `retrieval_context`

[Back to top](#title)

### Answer Relevancy
The answer relevancy metric measures the quality of your RAG pipeline's generator by evaluating how relevant the actual_output of your LLM application is compared to the provided `input`. `deepeval`'s answer relevancy metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score.

##### Required Arguments
To use the AnswerRelevancyMetric, you'll have to provide the following arguments when creating an LLMTestCase:

- `input`
- `actual_output`

[Back to top](#title)

### Hallucination
The hallucination metric determines whether your LLM generates factually correct information by comparing the `actual_output` to the provided `context`.

##### Required Arguments
To use the HallucinationMetric, you'll have to provide the following arguments when creating an LLMTestCase:

- `input`
- `actual_output`
- `retrieval_context`

[Back to top](#title)

-------------------

**Evaluate Redbox RAG chat on one stable, numbered version of these data**

**Set the version of the evaluation dataset you are using to evalute Redbox in the cell below**   <a class="anchor" id="setversion"></a>

In [3]:
DATA_VERSION = "0.1.0"

Run the cell below to set up the required folder structure (it will not overwrite folders and files if they already exist)

In [ ]:
from pathlib import Path

ROOT = Path.cwd().parents[1]
EVALUATION_DIR = ROOT / "notebooks/evaluation"

V_ROOT = EVALUATION_DIR / f"data/{DATA_VERSION}"
V_RAW = V_ROOT / "raw"
V_SYNTHETIC = V_ROOT / "synthetic"
V_CHUNKS = V_ROOT / "chunks"
V_RESULTS = V_ROOT / "results"

V_ROOT.mkdir(parents=True, exist_ok=True)
V_RAW.mkdir(parents=True, exist_ok=True)
V_SYNTHETIC.mkdir(parents=True, exist_ok=True)
V_CHUNKS.mkdir(parents=True, exist_ok=True)
V_RESULTS.mkdir(parents=True, exist_ok=True)

To save on API costs, we only need to generate a particular version of the evaluation dataset once. If you are using a previously generaterated evalutation dataset, please download it from shared team location (Google Drive).

---------

#### Imports <a id="imports"></a>

In [2]:
# Add autoreloatd
%reload_ext autoreload
%autoreload 2

In [ ]:
from jose import jwt
from uuid import UUID
import requests
import json
import pandas as pd

[Back to top](#title)

-----------------

## Start Redbox Running locally <a id="run-redbox"></a>

Run the following commands in terminal

Start docker runtime, example below uses colima
```bash
colima start --memory 8
``` 

-------------

#### First-time setup

First time users need to do the following

```bash
poetry install
```

Ensure you .env file has OpenAI API key in and has the following settings:

```bash
# === Object Storage ===

MINIO_HOST=minio
MINIO_PORT=9000
MINIO_ACCESS_KEY=minioadmin
MINIO_SECRET_KEY=minioadmin
AWS_ACCESS_KEY=minioadmin
AWS_SECRET_KEY=minioadmin

AWS_REGION=eu-west-2

# minio or s3
OBJECT_STORE=minio
BUCKET_NAME=redbox-storage-dev
```

Build redbox docker images (this takes several minutes)

```bash
docker compose build
```

------

If changes are made to the app, e.g. changes pulled in from main, it may require rebuilding docker images

```bash
docker compose build --no-cache
```

**Every time you start Redbox for evaluation (no Django frontend required), please run the following command**

```bash
make eval_backend
````

The above command will bring up everything you need for the backend (`core-api`, `worker`, `mino`, `elasticsearch` and `redis`), then create the MinIO bucket needed to store raw files

[Back to top](#title)

----------

## Generate `actual_output` using RAG and evaluation dataset

#### First need to upload files that we are going to 'RAG with'

**Use the printed out bearer token below to Authorize if you ever want to use the Swagger UI docs**

In [ ]:
bearer_token = jwt.encode({"user_uuid": str(UUID("aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa"))}, key="your-secret-key", algorithm="HS512")
print(bearer_token)

#### Get files that correspond to the version of evaluation dataset  <a class="anchor" id="files"></a>

Copy all the files that match your {DATA_VERSION} into `notebooks/evaluation/data/{DATA_VERSION}/raw/`. Find these files on the shared Google Drive and the corresponding version number/location

**It is really important to use the same files that were used to genearte this particular version of the evaluation dataset. A mismatch between the two will result in inaccurate evaluatoin metrics**

In [5]:
absolute_paths = [str(file.resolve()) for file in V_RAW.iterdir() if file.name != '.gitkeep']

**Only if you haven't uploaded files already** uncomment and run cell below

In [ ]:
# url = 'http://127.0.0.1:5002/file/upload'

# headers={
#     'accept': 'application/json',
#     "Authorization": f"Bearer {bearer_token}"
# }
# for file in absolute_paths:
#     files = {'file': open(file, 'rb')}
#     upload_file_response = requests.post(url, headers=headers, files=files)

#     #TODO: Add some login in the loop to deal with status codes != 200
#     # if upload_file_response.status_code != 200:
#     #     print("Failed to upload data:", upload_file_response.status_code)

List files uploaded to server

In [10]:
url = 'http://127.0.0.1:5002/file/'

headers={
    'accept': 'application/json',
    "Authorization": f"Bearer {bearer_token}"
}

file_list_response = requests.get(url, headers=headers)

View JSON response

In [11]:
if file_list_response.status_code == 200:
    # Parse JSON from the response
    data = file_list_response.json()
    
    # Pretty-print the JSON data
    pretty_json = json.dumps(data, indent=4)
    print(pretty_json)
else:
    print("Failed to retrieve data:", file_list_response.status_code)

[
    {
        "uuid": "f52a6c97-c234-40e5-a9af-94daab9035c1",
        "created_datetime": "2024-05-21T11:54:52.633856",
        "creator_user_uuid": "aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa",
        "key": "Frontier AI Taskforce_ second progress report - GOV.UK.pdf",
        "bucket": "redbox-storage-dev",
        "model_type": "File"
    },
    {
        "uuid": "6bb3be69-5b5b-4ad5-9623-0eb783d3502a",
        "created_datetime": "2024-05-21T11:54:52.707323",
        "creator_user_uuid": "aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa",
        "key": "Prime Minister's speech on AI_ 26 October 2023 - GOV.UK.pdf",
        "bucket": "redbox-storage-dev",
        "model_type": "File"
    },
    {
        "uuid": "961985cf-c5a1-4eeb-b472-25a06c8ef5dd",
        "created_datetime": "2024-05-21T11:54:52.778875",
        "creator_user_uuid": "aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa",
        "key": "The_impact_of_AI_on_UK_jobs_and_training_report.pdf",
        "bucket": "redbox-storage-dev",
        "model

Get a list of UUIDs

In [12]:
uuid_list = []

for item in data:
    if 'uuid' in item:
        uuid_list.append({'uuid': item['uuid']})

Get file status

In [ ]:
status_url_list = []
for uuid in uuid_list:
    url = f"http://127.0.0.1:5002/file/{uuid['uuid']}/status"
    status_url_list.append(url)

status_responses = []
for url in status_url_list:
    status_response = requests.get(url, headers=headers)
    status_responses.append(status_response)

for status in status_responses:
    data = status.json()
    pretty_json = json.dumps(data, indent=4)
    print(pretty_json)

------------

**Please ensure all emeddings have been completed before proceeding!**

Keep calm and go for a tea break!

--------------

#### Generate `actual_output` using `RAG` endpoint

In [12]:
df = pd.read_csv(f'{V_SYNTHETIC}/ragas_synthetic_data.csv', index=False)
inputs = df['input'].tolist()

In [13]:
retrieval_context = []
actual_output = []

headers = {
    'accept': 'application/json',
    'Authorization': 'Bearer ' + bearer_token,
    'Content-Type': 'application/json',
}

url = 'http://127.0.0.1:5002/chat/rag'

for input in inputs:
    data = {
        "message_history": [
            {
                "role": "system",
                "text": "You are a helpful AI Assistant"
            },
            {
                "role": "user",
                "text": input
            }
        ]
    }
    
    response = requests.post(url, headers=headers, data=json.dumps(data))
    data = response.json()

    retrieval_context.append(data['source_documents'])
    actual_output.append(data['output_text'])

df['actual_output'] = actual_output
df['retrieval_context'] = retrieval_context

In [ ]:
## Uncomment to check df now contains the actual_output and retrieved_context
# df.head()

#### Remove rows containing NaN to prevent Pydantic validation errors

In [45]:
df = df.dropna()
df.to_csv(f'{V_SYNTHETIC}/complete_ragas_synthetic_data.csv', index=False)

#### Validate evaluation data before running DeepEval metrics

[Back to top](#title)

----

In [55]:
df1 = pd.read_csv(f'{V_SYNTHETIC}/complete_ragas_synthetic_data.csv')

In [56]:
df1 = df1.iloc[0:2]

----------

## Load Evaluation Dataset into test cases <a class="anchor" id="load-test-cases"></a>

Put the CSV file that you want to use for evaluation into `/notebooks/evaluation/data/synthetic_data/` directory

Import test cases from CSV

In [48]:
from deepeval.dataset import EvaluationDataset

dataset = EvaluationDataset()
dataset.add_test_cases_from_csv_file(
    # file_path=f"./data/synthetic_data/complete_ragas_synthetic_data_v{version}.csv",
    file_path=f'{V_SYNTHETIC}/complete_ragas_synthetic_data.csv',
    input_col_name="input",
    actual_output_col_name="actual_output",
    expected_output_col_name="expected_output",
    context_col_name="context",
    context_col_delimiter= ";",
    retrieval_context_col_name="retrieval_context",
    retrieval_context_col_delimiter= ";"
)

[Back to top](#title)

---------

## Evaluate RAG pipeline <a id="evaluate"></a>

DeepEval imports

In [49]:
from deepeval import evaluate
from deepeval.metrics import (
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric,
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    HallucinationMetric,
)

Instantiate retrieval and generation evaluation metrics

In [50]:
# Instantiate retrieval metrics
contextual_precision = ContextualPrecisionMetric(
        threshold=0.5, # default is 0.5
    model="gpt-4o",
    include_reason=True
)

contextual_recall = ContextualRecallMetric(
    threshold=0.5, # default is 0.5
    model="gpt-4o",
    include_reason=True
)

contextual_relevancy = ContextualRelevancyMetric(
    threshold=0.5, # default is 0.5
    model="gpt-4o",
    include_reason=True
)

In [ ]:
# Instantiate generation metrics
answer_relevancy = AnswerRelevancyMetric(
    threshold=0.5, # default is 0.5
    model="gpt-4o",
    include_reason=True
)

faithfulness = FaithfulnessMetric(
    threshold=0.5, # default is 0.5
    model="gpt-4o",
    include_reason=True
)

hallucination = HallucinationMetric(
    threshold=0.5, # default is 0.5
    model="gpt-4o",
    include_reason=True
)

In [ ]:
dataset.test_cases

#### Retrieval Evaluation
Separate retrieval and generation evaluation results, as retrieval evalation can take some time

In [ ]:
retrieval_eval_results = evaluate(
    test_cases=dataset,
    metrics=[
        contextual_precision,
        contextual_recall,
        contextual_relevancy,
    ]
)

#### Save retrieval evaluation results

In [52]:
import pickle 
with open(f'{V_RESULTS}/retrieval_eval_results_v{DATA_VERSION}', 'wb') as f:
    pickle.dump(retrieval_eval_results, f)

#### Generation Evaluation

In [ ]:
generation_eval_results = evaluate(
    test_cases=dataset,
    metrics=[
        answer_relevancy,
        faithfulness,
        hallucination
    ]
)

#### Save generation evaluation results

In [83]:
import pickle 
with open(f'{V_RESULTS}/generation_eval_results_v{DATA_VERSION}', 'wb') as f:
    pickle.dump(generation_eval_results, f)

#### Access evaluation results from TestResult object

#### How to access metrics from TestResult object

In [48]:
generation_eval_results[0].success

True

In [49]:
generation_eval_results[0].metrics

In [53]:
generation_eval_results[0].metrics[0].score

1.0

In [54]:
generation_eval_results[0].metrics[0].reason

'The score is 1.00 because the response fully addresses the query about the impact of variations in UBI-like payments on childhood obesity rates without any irrelevant information. Great job on maintaining focus and relevancy!'

[Back to top](#title)

-------